In [4]:

d2=ConfigBox({'key':'value','key1':'value1'})

In [5]:
d2.key

'value'

creating data class (instad of creating python class where we need to define
the self keyword. we don't need to use in data class)

In [6]:
#update the entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    
    
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [7]:
from mlproject.constant import *
from mlproject.utils.common import read_yaml_file,create_directories

In [8]:
#update the configuration manager in src config
class ConfigurationManager:
    def __init__(self,
                config_filepath=config_file_path,
                params_filepath=config_file_path,
                schema_filepath=config_file_path):
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(params_filepath)
        self.schema=read_yaml_file(schema_filepath)
        
        create_directories([self.config.artifacts_root])  # this artifacts root name mentioned in the config.yaml
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        
        create_directories([self.config.artifacts_root])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        
        return data_ingestion_config
        

In [9]:
# update the components
import os 
import urllib.request as request
from mlproject import logger
import zipfile

class DataIngestion:
    def __init__(self,config=DataIngestionConfig):
        self.config=config
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            unzip_path=self.config.unzip_dir
            os.makedirs(unzip_path,exist_ok=True)
            filename,header=request.urlretrieve(
            url=self.config.source_url,
            filename=self.config.local_data_file)
            logger.info(f"{filename} download! with following info: \n {header}")
        else:
            logger.info(f"file already exists")
            

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
os.chdir("../")

In [11]:
try:
    config=ConfigurationManager()  ## reading the yaml file and passing the parameters/values
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-08-08 16:45:12,648: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-08-08 16:45:12,654: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-08-08 16:45:12,666: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-08-08 16:45:12,669: INFO: common: created directory at artifacts:]
[2024-08-08 16:45:12,671: INFO: common: created directory at artifacts:]
[2024-08-08 16:45:13,868: INFO: 416729189: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2D44:17E4F6:551027:6E0658:66B4A849
Accept-Ranges: bytes
Date: Thu, 